In [ ]:
import numpy as np;
import pandas as pd;
import plotly.express as px;
import json

## Initialize Parameters

In [2]:
# Global parameters
NO_OF_DAYS = 180
modelParameter = {}
modelParameter["pi"] = 0.02
modelParameter["tE"] = 3
modelParameter["tI"] = 7
modelParameter["cw"] = 15
modelParameter["ch"] = 5
modelParameter["eBC"] = 2/3
modelParameter["eLD"] = 1/10
modelParameter["eDB"] = 1/5

district = "all"

print(modelParameter)


{'pi': 0.02, 'tE': 3, 'tI': 7, 'cw': 15, 'ch': 5, 'eBC': 0.6666666666666666, 'eLD': 0.1, 'eDB': 0.2}


## Load and Initialize Data

In [3]:
## Population data
initDataDF = pd.read_csv("../data/" + district + "_init_data.csv")
popDataDF = pd.read_csv("../data/" + district + "_population_data.csv")


cSparseMatrix = pd.read_csv("../output/" + district + "_cmatrix_results.csv")

## Number of regions
r = len(initDataDF.index)

## cijMatrix
C_TPart_Matrix = cSparseMatrix['Cij'].to_numpy().reshape(r,r)

## Sort distance data by name for easy matrix transformation
initDataDF.sort_values(by=['name'], inplace=True)
popDataDF.sort_values(by=['name'], inplace=True)

initDataDF['N'] = popDataDF['N']
initDataDF['type'] = popDataDF['type']

Wn = modelParameter["cw"]*C_TPart_Matrix
Wn

array([[1.28101813e+01, 7.97311329e-04, 4.84355174e-05, ...,
        8.49901483e-04, 1.85589615e-04, 3.13397084e-05],
       [1.82987850e-04, 1.28299576e+01, 7.36349206e-05, ...,
        2.10398700e-04, 8.58909966e-05, 5.03575492e-05],
       [1.05660214e-05, 6.99901693e-05, 1.26325617e+01, ...,
        1.62915671e-05, 2.29139708e-05, 4.34002239e-03],
       ...,
       [6.09100213e-04, 6.57005336e-04, 5.35223885e-05, ...,
        1.40700547e+01, 1.86632041e-04, 3.40275355e-05],
       [4.64543796e-05, 9.36755640e-05, 2.62921322e-05, ...,
        6.51837040e-05, 1.39724702e+01, 1.58715701e-05],
       [1.26524417e-05, 8.85828522e-05, 8.03200226e-03, ...,
        1.91685704e-05, 2.55991940e-05, 1.26165216e+01]])

## Mitigation Strategy initialization

### 1. Normal Day

In [4]:
Mn = np.ones((r,r))
Mn

array([[1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       ...,
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.]])

### 2. Break the chain

In [5]:
Mbc = Mn*modelParameter["eBC"]
Mbc

array([[0.66666667, 0.66666667, 0.66666667, ..., 0.66666667, 0.66666667,
        0.66666667],
       [0.66666667, 0.66666667, 0.66666667, ..., 0.66666667, 0.66666667,
        0.66666667],
       [0.66666667, 0.66666667, 0.66666667, ..., 0.66666667, 0.66666667,
        0.66666667],
       ...,
       [0.66666667, 0.66666667, 0.66666667, ..., 0.66666667, 0.66666667,
        0.66666667],
       [0.66666667, 0.66666667, 0.66666667, ..., 0.66666667, 0.66666667,
        0.66666667],
       [0.66666667, 0.66666667, 0.66666667, ..., 0.66666667, 0.66666667,
        0.66666667]])

### 3. Complete Lockdown


In [6]:
Mld = Mn*modelParameter["eLD"]
Mld

array([[0.1, 0.1, 0.1, ..., 0.1, 0.1, 0.1],
       [0.1, 0.1, 0.1, ..., 0.1, 0.1, 0.1],
       [0.1, 0.1, 0.1, ..., 0.1, 0.1, 0.1],
       ...,
       [0.1, 0.1, 0.1, ..., 0.1, 0.1, 0.1],
       [0.1, 0.1, 0.1, ..., 0.1, 0.1, 0.1],
       [0.1, 0.1, 0.1, ..., 0.1, 0.1, 0.1]])

### 4. District Border Closure

In [7]:
cSparseMatrix['iNameDistrict'] = cSparseMatrix['iName'].str.split("__",n = 2, expand = True)[1] 
cSparseMatrix['jNameDistrict'] = cSparseMatrix['jName'].str.split("__", n = 2, expand = True)[1] 

cSparseMatrix.loc[cSparseMatrix['iNameDistrict']==cSparseMatrix['jNameDistrict'],'interDistrictFlag'] = 0
cSparseMatrix.loc[cSparseMatrix['iNameDistrict']!=cSparseMatrix['jNameDistrict'],'interDistrictFlag'] = 1


epsilonDistrictLockDownMatrix = cSparseMatrix['interDistrictFlag'].to_numpy().reshape(r,r)*modelParameter["eDB"]
Mdb = np.multiply(Mn,epsilonDistrictLockDownMatrix)
Mdb[epsilonDistrictLockDownMatrix==0] = 1
Mdb

array([[1. , 0.2, 0.2, ..., 1. , 0.2, 0.2],
       [0.2, 1. , 0.2, ..., 0.2, 0.2, 0.2],
       [0.2, 0.2, 1. , ..., 0.2, 0.2, 1. ],
       ...,
       [1. , 0.2, 0.2, ..., 1. , 0.2, 0.2],
       [0.2, 0.2, 0.2, ..., 0.2, 1. , 0.2],
       [0.2, 0.2, 1. , ..., 0.2, 0.2, 1. ]])

## User parameters

In [8]:
userParameter = '{ "breakTheChains" : [{"startDay": 7, "endDay": 49}], "completeLockDowns" :  [{"startDay": 14, "endDay": 35}], "districtLockDowns" :  [{"startDay": 30, "endDay": 49}]}'
                

userParameterData = json.loads(userParameter)
userParameterData



{'breakTheChains': [{'startDay': 7, 'endDay': 49}],
 'completeLockDowns': [{'startDay': 14, 'endDay': 35}],
 'districtLockDowns': [{'startDay': 30, 'endDay': 49}]}

In [9]:
def isBreakTheChain(day):
    for breakTheChains in userParameterData['breakTheChains']:
        if day >= breakTheChains['startDay'] and day <= breakTheChains['endDay']:
            return True
    return False

In [10]:
def isCompleteLockdown(day):
    for completeLockDowns in userParameterData['completeLockDowns']:
        if day >= completeLockDowns['startDay'] and day <= completeLockDowns['endDay']:
            return True
    return False


In [11]:
def isDistrictLockDown(day):
    for districtLockDowns in userParameterData['districtLockDowns']:
        if day >= districtLockDowns['startDay'] and day <= districtLockDowns['endDay']:
            return True
    return False


## Function definitions

In [12]:
## Differential equation
def deriv(y,contactRatio, modelParameter):
    Si, Ei, Ii, Ri = y
    dSdt = -modelParameter["pi"] * Si * contactRatio
    dEdt = (modelParameter["pi"] * Si * contactRatio) - (Ei/modelParameter["tE"])
    dIdt = (Ei/modelParameter["tE"]) -  (Ii/modelParameter["tI"])
    dRdt = (Ii/modelParameter["tI"])
    return dSdt, dEdt, dIdt, dRdt

In [21]:
def computeContactMatrix(day):    
    ## Effective mitigation matrix
    Meffective = Mn
    
    if isBreakTheChain(day):
        Meffective = np.minimum(Meffective, Mbc)
        
    if isCompleteLockdown(day):
        Meffective = np.minimum(Meffective, Mld)
    
    if isDistrictLockDown(day):
        Meffective = np.minimum(Meffective, Mdb)
        
    Cmitigated = np.multiply(Meffective,Wn)    
    np.fill_diagonal(Cmitigated, Cmitigated.diagonal() + modelParameter["ch"])
    return Cmitigated
    


In [22]:
## Find the contact ratio
def findSpatialContactRatioV2(Ci, I, N, day):
    c = Ci*I/N
    return c.sum()

In [23]:
## derive next count for SEIHR compartments
def derivNext(y, rates) :
    S,E,I,R = y
    dSdt, dEdt, dIdt, dRdt = rates
    S1 = S + dSdt
    E1 = E + dEdt
    I1 = I + dIdt
    R1 = R + dRdt

    return S1, E1, I1, R1


## Run the model for all regions

In [24]:
yPrevDict = {};
yNextDict = {};
data = []
IPrevArr = []
NArray = []


for index, row in initDataDF.iterrows():
    yPrevDict[row['name']] = row['S'], row['E'], row['I'], row['R'] 
    IPrevArr.append(row['I'])
    NArray.append(row['S'] + row['E'] + row['I'] + row['R'])

IPrev = np.array(IPrevArr);
N = np.array(NArray);


for day in range(1,NO_OF_DAYS):
    INextArr = [];
    C = computeContactMatrix(day)

    for index, row in initDataDF.iterrows():
        name = row['name'];
        y = yPrevDict[name]; 
        # Contact matrix
        contactRatio = findSpatialContactRatioV2(C[index],IPrev, N, day);
        rates = deriv(y, contactRatio, modelParameter)
        yNext = derivNext(y, rates) 
        
        yNextDict[name] = yNext
        data.append({'name': name, 'day': day, 'S': yNext[0], 'E': yNext[1], 'I': yNext[2], 'R': yNext[3]})
        INextArr.append(yNext[2])
    
    yPrevDict = yNextDict
    IPrev = np.array(INextArr);

resultsDF = pd.DataFrame(data);



In [ ]:
resultsDF.head(10)

## Save Output

In [ ]:
resultsDF.to_csv("../output/" + district + "_results.csv")

## Plot Results

In [ ]:
plotsDF = resultsDF.copy()
plotsDF['TotalI'] = plotsDF['I']
plotsDF['day'] = plotsDF['day']
plotsDF['name'] = plotsDF['name']
plotsDF = plotsDF.drop(plotsDF[plotsDF['TotalI']==0].index,axis=0)

fig = px.scatter(plotsDF, x="day", y="TotalI", color="name")
fig.show()

In [ ]:
import plotly.graph_objects as go
figGo = go.Figure()

for index, row in initDataDF.iterrows(): 
    figGo.add_trace(go.Scatter(
    x=resultsDF[resultsDF['name']==row['name']]['day'],
    y=resultsDF[resultsDF['name']==row['name']]['I'],
    name=row['name']))


figGo.update_layout(yaxis_type="log")
figGo.show()

In [ ]:
idx = resultsDF.groupby(['name'])['I'].transform(max) == resultsDF['I']
resultsDF[idx].to_csv("../output/" + district + "_peak_results.csv")
resultsDF[idx]